In [4]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit_ibm_catalog import QiskitServerless, QiskitFunction
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import SabreLayout, SabreSwap
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
import matplotlib.pyplot as plt
import numpy as np
import time

## Creating a Star-Oriented GHZ Circuit

It's a star to make SABREs life difficult :)

In [8]:
# set seed for reproducibility
seed = 42
num_qubits = 50
 
# Create GHZ circuit
qc = QuantumCircuit(num_qubits)
qc.h(0)
for i in range(1, num_qubits):
    qc.cx(0, i)
 
qc.measure_all()

In [6]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (num_qubits - 2 - i)
    for i in range(num_qubits - 1)
]
# print(operator_strings)
# print(len(operator_strings))
 
operators = [SparsePauliOp(operator) for operator in operator_strings]

In [7]:
from qiskit import QuantumCircuit, transpile

# 1. Your high-level circuit
qc = QuantumCircuit(3)
qc.h(0)
qc.swap(0, 2)  # SWAP is rarely a native gate
qc.cx(0, 1)

# 2. Define the Target Native Gates
# Common for Superconducting (IBM): ['rz', 'sx', 'x', 'ecr'] or ['rz', 'sx', 'x', 'cx']
# Common for Ion Traps: ['rxx', 'rz']
native_gates = ['r', 'rz', 'cx'] 

# 3. Transpile
# optimization_level=3 tries hardest to reduce gate count
transpiled_qc = transpile(qc, basis_gates=native_gates, optimization_level=3)

print("Original Ops:", qc.count_ops())
print("Native Ops:", transpiled_qc.count_ops())
print(transpiled_qc.draw())

Original Ops: OrderedDict([('h', 1), ('swap', 1), ('cx', 1)])
Native Ops: OrderedDict([('r', 2), ('cx', 1)])
global phase: π/2
         ┌────────────┐┌────────┐
q_0 -> 0 ┤ R(π/2,π/2) ├┤ R(π,0) ├
         └───┬───┬────┘└────────┘
q_1 -> 1 ────┤ X ├───────────────
             └─┬─┘               
q_2 -> 2 ──────■─────────────────
                                 
